In [69]:
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [70]:
# TRAIN_PATH = './asl_alphabet_train/asl_alphabet_train'
# TEST_PATH = './asl_alphabet_test/asl_alphabet_test/'
# PATH = './myData/SIBI'
PATH = './myData/BISINDO'
TARGET_SHAPE = 200

In [71]:
train_datagen = ImageDataGenerator(     
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    rescale = 1./255,
    validation_split = 0.25 )

test_datagen  = ImageDataGenerator(         
    rescale = 1./255,
    validation_split = 0.25)

train_dataset = train_datagen.flow_from_directory(
    PATH,
    target_size=(TARGET_SHAPE,TARGET_SHAPE),
    batch_size=16,
    class_mode = 'categorical',
    subset = 'training'
)

valid_dataset = test_datagen.flow_from_directory(
    PATH,
    target_size=(TARGET_SHAPE,TARGET_SHAPE),
    batch_size=16,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 751 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [72]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
import cv2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [73]:
from tensorflow.keras.applications import VGG16,MobileNetV2

In [74]:
# model = VGG16(input_shape=(TARGET_SHAPE,TARGET_SHAPE,3),weights='imagenet',include_top=False)
# model.summary()
model = MobileNetV2(input_shape=(TARGET_SHAPE,TARGET_SHAPE,3),weights='imagenet',include_top=False)
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 100, 100, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 100, 100, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [75]:
model.output

<KerasTensor: shape=(None, 7, 7, 1280) dtype=float32 (created by layer 'out_relu')>

In [76]:
classes = 5

x = model.output
x = tf.keras.layers.Convolution2D(classes,(1,1),name='predictions')(x)
x = Flatten()(x)
x = tf.keras.layers.Dense(classes)(x)
predictions = tf.keras.layers.Dense(classes,activation='softmax')(x)
# predictions

In [77]:
real_model = tf.keras.models.Model(inputs = model.input, outputs=predictions)
for layer in model.layers:
    layer.trainable = False

In [78]:
real_model.compile( 
    optimizer = Adam(learning_rate = 0.0001), 
    loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics=['accuracy']
)

In [79]:
history = real_model.fit(x = train_dataset,epochs = 20,validation_data = valid_dataset,verbose=1,batch_size=64)

Epoch 1/20
47/47 [==============================] - 22s 417ms/step - loss: 2.7687 - accuracy: 0.2743 - val_loss: 1.9106 - val_accuracy: 0.3440
Epoch 2/20
47/47 [==============================] - 19s 401ms/step - loss: 1.6680 - accuracy: 0.4234 - val_loss: 1.2485 - val_accuracy: 0.5240
Epoch 3/20
47/47 [==============================] - 19s 393ms/step - loss: 1.1410 - accuracy: 0.5832 - val_loss: 0.8415 - val_accuracy: 0.6760
Epoch 4/20
47/47 [==============================] - 20s 417ms/step - loss: 0.8666 - accuracy: 0.6698 - val_loss: 0.5541 - val_accuracy: 0.7720
Epoch 5/20
47/47 [==============================] - 19s 394ms/step - loss: 0.6377 - accuracy: 0.7630 - val_loss: 0.4201 - val_accuracy: 0.8480
Epoch 6/20
47/47 [==============================] - 20s 419ms/step - loss: 0.5002 - accuracy: 0.8083 - val_loss: 0.3003 - val_accuracy: 0.9040
Epoch 7/20
47/47 [==============================] - 20s 416ms/step - loss: 0.3851 - accuracy: 0.8482 - val_loss: 0.2587 - val_accuracy: 0.9040

In [84]:
real_model.save('my_model4.h5')

In [85]:
train_dataset.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

In [82]:
# import pathlib

# # Convert SavedModel menjadi vegs.tflite
# converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# tflite_model = converter.convert()

# tflite_model_file = pathlib.Path('vegs.tflite')
# tflite_model_file.write_bytes(tflite_model)

In [83]:
# export_dir = 'saved_model/'
# converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# converter.optimizations = [tf.lite.Optimize.DEFAULT] #ngecilin ukuran model
# # converter.representative_dataset = representative_dataset
# tflite_model = converter.convert()

# tflite_model_file = pathlib.Path('model2.tflite')
# tflite_model_file.write_bytes(tflite_model)